In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_columns',5000)
pd.set_option('display.max_rows',5000)

import lxml
import html5lib
from urllib.request import urlopen
import time

from bs4 import BeautifulSoup
import requests

# Get Raw Batting Data
- Currently the model averages the recent team hitting performance
- This does not account for the particular players in the starting lineup that day

E.g. If a key hitter is resting, injured, got traded, etc.
To begin to model this we first need to scrape the raw batter data (similar to how we got the pitching data)

In [2]:
def get_daily_season_links_batter(batter_id):
    letter = batter_id.upper()[0]
    url_prefix = 'https://www.retrosheet.org/boxesetc/'
    url = url_prefix+letter+'/P'+batter_id+'.htm'
    time.sleep(.5)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    html=list(soup.children)


    body = list(html[2].children)[5]
    pre_texts = [x for x in body.find_all('pre')]
    secnum = np.where([x.get_text().strip().startswith('Batting Record') for x in pre_texts])[0][0]
    a_pre_texts = pre_texts[secnum].find_all('a')
    daily_season_links = [url_prefix+x.attrs['href'][3:] for x in a_pre_texts if x.get_text()=='Daily']
    return(daily_season_links)

In [3]:
get_daily_season_links_batter('hernk001')

['https://www.retrosheet.org/boxesetc/1974/Ihernk0010011974.htm',
 'https://www.retrosheet.org/boxesetc/1975/Ihernk0010021975.htm',
 'https://www.retrosheet.org/boxesetc/1976/Ihernk0010031976.htm',
 'https://www.retrosheet.org/boxesetc/1977/Ihernk0010041977.htm',
 'https://www.retrosheet.org/boxesetc/1978/Ihernk0010051978.htm',
 'https://www.retrosheet.org/boxesetc/1979/Ihernk0010061979.htm',
 'https://www.retrosheet.org/boxesetc/1980/Ihernk0010071980.htm',
 'https://www.retrosheet.org/boxesetc/1981/Ihernk0010081981.htm',
 'https://www.retrosheet.org/boxesetc/1982/Ihernk0010091982.htm',
 'https://www.retrosheet.org/boxesetc/1983/Ihernk0010101983.htm',
 'https://www.retrosheet.org/boxesetc/1983/Ihernk0010111983.htm',
 'https://www.retrosheet.org/boxesetc/1984/Ihernk0010121984.htm',
 'https://www.retrosheet.org/boxesetc/1985/Ihernk0010131985.htm',
 'https://www.retrosheet.org/boxesetc/1986/Ihernk0010141986.htm',
 'https://www.retrosheet.org/boxesetc/1987/Ihernk0010151987.htm',
 'https://

In [3]:
def get_season_batting_data(url):    
    time.sleep(.3)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    html=list(soup.children)[-1]
    body = list(html.children)[-1]
    sec_next = list(body.children)
    secnum = np.where(["Opponent" in str(x) for x in sec_next])[0][0]
    key_section = sec_next[secnum]
    working_part = list(key_section.children)
    p_header = working_part[0].strip().split()
    mod_header= ['at_vs','Opponent','League', 'GS', 'AB', 'R', 'H', '2B', '3B', 'HR',
       'RBI', 'BB', 'IBB', 'SO', 'HBP', 'SH', 'SF', 'XI', 'ROE', 'GDP',
       'SB', 'CS', 'AVG', 'OBP', 'SLG', 'BP', 'Pos']

    date_list = []
    day_href_list = []
    for k in range(1,len(working_part),4):
        date_list.append(working_part[k].get_text().strip())
        day_href_list.append(working_part[k].attrs['href'])

    dblhead_num_list = []
    for k in range(2,len(working_part),4):
        dblhead_num_list.append(working_part[k].strip())

    game_href_list = []
    for k in range(3,len(working_part),4):
        game_href_list.append(working_part[k].attrs['href'])

    main_data_matrix = []
    for k in range(4,len(working_part),4):
        main_data_row = (working_part[k].strip().split())[:27]
        main_data_matrix.append(main_data_row)
    row_sizes = [len(x) for x in main_data_matrix]
    max_row_size = max(row_sizes)
    min_row_size = min(row_sizes)
    if (min_row_size == max_row_size) and (max_row_size==27):
        # Everything has all 27 columns
        out_df = pd.DataFrame(main_data_matrix, columns = mod_header)
    elif (min_row_size == max_row_size) and (max_row_size==26):
        # Everything has 26 columns, will guess position is missing
        out_df = pd.DataFrame(main_data_matrix, columns = mod_header[:26])
        out_df['Pos'] = ''
    elif (min_row_size == 26) and (max_row_size==27):
        # Guessing position is missing for some rows but not others
        main_data_matrix = [x if len(x)==27 else x+[''] for x in main_data_matrix]
        out_df = pd.DataFrame(main_data_matrix, columns = mod_header)
    else:
        print('finding rows with less than 26 or more than 27 entries - Returning None')
        return(None)
    out_df['Date'] = date_list
    out_df['dblhead_num'] = dblhead_num_list
    return(out_df)

In [5]:
get_season_batting_data('https://www.retrosheet.org/boxesetc/1986/Ihernk0010141986.htm')

,at_vs,Opponent,League,GS,AB,R,H,2B,3B,HR,RBI,BB,IBB,SO,HBP,SH,SF,XI,ROE,GDP,SB,CS,AVG,OBP,SLG,BP,Pos,Date,dblhead_num
0,AT,PIT,N,1,4,1,2,1,0,0,2,1,0,0,0,0,0,0,0,0,0,0,.500,.600,.750,3,1b,4- 8-1986,
1,AT,PHI,N,1,5,3,2,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,.444,.500,.667,3,1b,4-11-1986,
2,AT,PHI,N,1,7,0,0,0,0,0,0,0,0,1,1,0,0,0,0,2,0,0,.250,.333,.375,3,1b,4-12-1986,
3,AT,PHI,N,1,3,1,3,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,.368,.455,.526,3,1b,4-13-1986,
4,VS,STL,N,1,4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,.348,.423,.478,3,1b,4-14-1986,
5,VS,PHI,N,1,3,2,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,.346,.433,.462,3,1b,4-18-1986,
6,VS,PHI,N,1,4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,.333,.412,.433,3,1b,4-19-1986,
7,VS,PHI,N,1,3,1,1,0,0,0,1,2,0,2,0,0,0,0,0,0,0,0,.333,.436,.424,3,1b,4-20-1986,
8,VS,PIT,N,1,2,1,0,0,0,0,0,2,0,0,1,0,0,0,0,0,0,0,.314,.455,.400,3,1b,4-21-1986,
9,VS,PIT,N,1,5,0,1,1,0,0,2,0,0,1,0,0,0,0,0,0,0,0,.300,.429,.400,3,1b,4-22-1986,


In [4]:
# Get all the data for a particular batter
def get_full_batting_data(batter_id):
    link_list = get_daily_season_links_batter(batter_id)
    df_batting = pd.DataFrame()
    for url in link_list:
        df_batting = pd.concat((df_batting, get_season_batting_data(url)))
    return(df_batting)

In [7]:
df_keith = get_full_batting_data('hernk001')

In [11]:
df_keith.sample(5)

,at_vs,Opponent,League,GS,AB,R,H,2B,3B,HR,RBI,BB,IBB,SO,HBP,SH,SF,XI,ROE,GDP,SB,CS,AVG,OBP,SLG,BP,Pos,Date,dblhead_num
88,VS,PHI,N,1,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,.274,.334,.413,3,1b,9-22-1988,
8,AT,STL,N,1,2,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,1,.206,.308,.265,5,1b,4-16-1989,
82,VS,SD,N,1,4,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,.320,.402,.484,3,1b,7-15-1980,
101,AT,CIN,N,1,2,2,1,0,0,1,1,2,0,1,0,0,0,0,0,0,0,0,.308,.399,.477,5,1b,7-30-1977,
38,VS,PIT,N,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,.268,.340,.377,9,ph,7-27-1989,


In [5]:
df = pd.read_csv('Data/df_bp7.csv')
df.shape

/tmp/ipykernel_104482/1191105669.py:1: DtypeWarning: Columns (13,14,15,19,85,87,159,198) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Data/df_bp7.csv')


(96250, 611)

In [13]:
df.head()

,date,dblheader_code,day_of_week,team_v,league_v,game_no_v,team_h,league_h,game_no_h,runs_v,runs_h,outs_total,day_night,completion_info,forfeit_info,protest_info,ballpark_id,attendance,game_minutes,linescore_v,linescore_h,AB_v,H_v,2B_v,3B_v,HR_v,RBI_v,SH_v,SF_v,HBP_v,BB_v,IBB_v,SO_v,SB_v,CS_v,GIDP_v,CI_v,LOB_v,P_num_v,ERind_v,ERteam_v,WP_v,balk_v,PO_v,ASST_v,ERR_v,PB_v,DP_v,TP_v,AB_h,H_h,2B_h,3B_h,HR_h,RBI_h,SH_h,SF_h,HBP_h,BB_h,IBB_h,SO_h,SB_h,CS_h,GIDP_h,CI_h,LOB_h,P_num_h,ERind_h,ERteam_h,WP_h,balk_h,PO_h,ASST_h,ERR_h,PB_h,DP_h,TP_h,ump_HB_id,ump_HB_name,ump_1B_id,ump_1B_name,ump_2B_id,ump_2B_name,ump_3B_id,ump_3B_name,ump_LF_id,ump_LF_name,ump_RF_id,ump_RF_name,mgr_id_v,mgr_name_v,mgr_id_h,mgr_name_h,pitcher_id_w,pitcher_name_w,pitcher_id_l,pitcher_name_l,pitcher_id_s,pitcher_name_s,GWRBI_id,GWRBI_name,pitcher_start_id_v,pitcher_start_name_v,pitcher_start_id_h,pitcher_start_name_h,batter1_name_v,batter1_id_v,batter1_pos_v,batter2_name_v,batter2_id_v,batter2_pos_v,batter3_name_v,batter3_id_v,batter3_pos_v,batter4_name_v,batter4_id_v,batter4_pos_v,batter5_name_v,batter5_id_v,batter5_pos_v,batter6_name_v,batter6_id_v,batter6_pos_v,batter7_name_v,batter7_id_v,batter7_pos_v,batter8_name_v,batter8_id_v,batter8_pos_v,batter9_name_v,batter9_id_v,batter9_pos_v,batter1_name_h,batter1_id_h,batter1_pos_h,batter2_name_h,batter2_id_h,batter2_pos_h,batter3_name_h,batter3_id_h,batter3_pos_h,batter4_name_h,batter4_id_h,batter4_pos_h,batter5_name_h,batter5_id_h,batter5_pos_h,batter6_name_h,batter6_id_h,batter6_pos_h,batter7_name_h,batter7_id_h,batter7_pos_h,batter8_name_h,batter8_id_h,batter8_pos_h,batter9_name_h,batter9_id_h,batter9_pos_h,misc_info,acqui_info,season,run_diff,home_victory,run_total,date_dblhead,BATAVG_162_h,BATAVG_162_v,OBP_162_h,OBP_162_v,SLG_162_h,SLG_162_v,OBS_162_h,OBS_162_v,SB_162_h,SB_162_v,CS_162_h,CS_162_v,ERR_162_h,ERR_162_v,BATAVG_30_h,BATAVG_30_v,OBP_30_h,OBP_30_v,SLG_30_h,SLG_30_v,OBS_30_h,OBS_30_v,SB_30_h,SB_30_v,CS_30_h,CS_30_v,ERR_30_h,ERR_30_v,implied_prob_h,implied_prob_v,implied_prob_h_mid,over_under_line,over_under_result,Strt_GS_h,Strt_GS_v,Strt_IP_h,Strt_IP_v,Strt_H_h,Strt_H_v,Strt_BFP_h,Strt_BFP_v,Strt_HR_h,Strt_HR_v,Strt_R_h,Strt_R_v,Strt_ER_h,Strt_ER_v,Strt_BB_h,Strt_BB_v,Strt_IB_h,Strt_IB_v,Strt_SO_h,Strt_SO_v,Strt_SH_h,Strt_SH_v,Strt_SF_h,Strt_SF_v,Strt_WP_h,Strt_WP_v,Strt_HBP_h,Strt_HBP_v,Strt_BK_h,Strt_BK_v,Strt_2B_h,Strt_2B_v,Strt_3B_h,Strt_3B_v,Strt_IP_real_h,Strt_IP_real_v,Strt_rollsum_IP_real_10_h,Strt_rollsum_IP_real_10_v,Strt_rollsum_H_10_h,Strt_rollsum_H_10_v,Strt_rollsum_BFP_10_h,Strt_rollsum_BFP_10_v,Strt_rollsum_HR_10_h,Strt_rollsum_HR_10_v,Strt_rollsum_R_10_h,Strt_rollsum_R_10_v,Strt_rollsum_ER_10_h,Strt_rollsum_ER_10_v,Strt_rollsum_BB_10_h,Strt_rollsum_BB_10_v,Strt_rollsum_IB_10_h,Strt_rollsum_IB_10_v,Strt_rollsum_SO_10_h,Strt_rollsum_SO_10_v,Strt_rollsum_SH_10_h,Strt_rollsum_SH_10_v,Strt_rollsum_SF_10_h,Strt_rollsum_SF_10_v,Strt_rollsum_WP_10_h,Strt_rollsum_WP_10_v,Strt_rollsum_HBP_10_h,Strt_rollsum_HBP_10_v,Strt_rollsum_BK_10_h,Strt_rollsum_BK_10_v,Strt_rollsum_2B_10_h,Strt_rollsum_2B_10_v,Strt_rollsum_3B_10_h,Strt_rollsum_3B_10_v,Strt_rollsum_IP_real_35_h,Strt_rollsum_IP_real_35_v,Strt_rollsum_H_35_h,Strt_rollsum_H_35_v,Strt_rollsum_BFP_35_h,Strt_rollsum_BFP_35_v,Strt_rollsum_HR_35_h,Strt_rollsum_HR_35_v,Strt_rollsum_R_35_h,Strt_rollsum_R_35_v,Strt_rollsum_ER_35_h,Strt_rollsum_ER_35_v,Strt_rollsum_BB_35_h,Strt_rollsum_BB_35_v,Strt_rollsum_IB_35_h,Strt_rollsum_IB_35_v,Strt_rollsum_SO_35_h,Strt_rollsum_SO_35_v,Strt_rollsum_SH_35_h,Strt_rollsum_SH_35_v,Strt_rollsum_SF_35_h,Strt_rollsum_SF_35_v,Strt_rollsum_WP_35_h,Strt_rollsum_WP_35_v,Strt_rollsum_HBP_35_h,Strt_rollsum_HBP_35_v,Strt_rollsum_BK_35_h,Strt_rollsum_BK_35_v,Strt_rollsum_2B_35_h,Strt_rollsum_2B_35_v,Strt_rollsum_3B_35_h,Strt_rollsum_3B_35_v,Strt_rollsum_IP_real_75_h,Strt_rollsum_IP_real_75_v,Strt_rollsum_H_75_h,Strt_rollsum_H_75_v,Strt_rollsum_BFP_75_h,Strt_rollsum_BFP_75_v,Strt_rollsum_HR_75_h,Strt_rollsum_HR_75_v,Strt_rol

In [6]:
batter_ids = np.array([])
for num in range(1,10):
    for suffix in ['_h','_v']:
        # Check whether this should be '_id' or '_name'
        colname = 'batter'+str(num)+'_name'+suffix
        batter_ids = np.concatenate((batter_ids, pd.unique(df[colname])))
batter_ids = pd.unique(batter_ids)

In [7]:
len(batter_ids), batter_ids[:10]

(6459,
 array(['cruzj002', 'colld001', 'molib101', 'wilsw001', 'molip001',
        'hendr001', 'rivem001', 'puhlt001', 'tavef101', 'smito001'],
       dtype=object))

In [17]:
for i in range(1): # for everyone, do range(len(batter_ids))
    p_id = batter_ids[i]
    print(i, p_id)
    df_temp = get_full_batting_data(p_id)
    # put path here to save to a specific folder
    fname_out = 'batting_data_'+p_id+'.csv'
    df_temp.to_csv(fname_out, index=False)

0 cruzj002


In [8]:

test_batch = batter_ids[10:15]
batch_1 = batter_ids[1610:2500]
batch_2 = batter_ids[2500:4000]
batch_3 = batter_ids[4000:5500]
batch_4 = batter_ids[5500:]

def run_batch(Batch):
    n = len(Batch)
    print("Starting..\n")
    t = time.time()
    for p_id in Batch:
        try:
            print(f"Getting P_ID: {p_id}")
            df_temp = get_full_batting_data(p_id)
            
            fname_out = 'RS-Batters/batting_data_'+p_id+'.csv'
            df_temp.to_csv(fname_out, index=False)
        except:
            print(f"Error: {p_id}")
            
    s = int(time.time() - t)
    m = round(s/60, 2)
    p = round(s/n, 2)
    print(f"\nCompleted in: {m} mins\n{p} secs/batter")

In [ ]:
#run_batch(test_batch)

In [9]:
run_batch(batch_1)

Starting..

Getting P_ID: smitp002
Getting P_ID: taylt002
Getting P_ID: kellc002
Getting P_ID: taucm001
Getting P_ID: tom-k001
Getting P_ID: kelej001
Getting P_ID: madrn001
Getting P_ID: garcr005
Getting P_ID: garlk001
Getting P_ID: wardt002
Getting P_ID: rojaj002
Getting P_ID: uptoj001
Getting P_ID: sotoj001
Getting P_ID: suare001
Getting P_ID: indij001
Getting P_ID: larnt001
Getting P_ID: guill001
Getting P_ID: carld002
Getting P_ID: badda001
Getting P_ID: willl001
Getting P_ID: adrie001
Getting P_ID: white006
Getting P_ID: hilld002
Getting P_ID: mccoc001
Getting P_ID: parkh001
Getting P_ID: refsr001
Getting P_ID: herny003
Getting P_ID: joe-c001
Getting P_ID: hermm001
Getting P_ID: thoml002
Getting P_ID: ruf-d001
Getting P_ID: hayek001
Getting P_ID: mottt001
Getting P_ID: vierm001
Getting P_ID: marsb002
Getting P_ID: mcker001
Getting P_ID: nootl001
Getting P_ID: sirij001
Getting P_ID: millb003
Getting P_ID: rosae001
Getting P_ID: hummc001
Getting P_ID: voged001
Getting P_ID: nolaa002

In [10]:
run_batch(batch_2)

Starting..

Getting P_ID: arcio002
Getting P_ID: rivet001
Getting P_ID: hoodd001
Getting P_ID: scrux001
Getting P_ID: selss001
Getting P_ID: gurry001
Getting P_ID: eibnb001
Getting P_ID: castn001
Getting P_ID: stasb001
Getting P_ID: sancg002
Getting P_ID: avila001
Getting P_ID: bonij002
Getting P_ID: smitt002
Getting P_ID: camaj001
Getting P_ID: olsom001
Getting P_ID: osunj001
Getting P_ID: gomem002
Getting P_ID: adduj002
Getting P_ID: joner005
Getting P_ID: sanca007
Getting P_ID: kivlp001
Getting P_ID: blasj001
Getting P_ID: telit001
Getting P_ID: mejia002
Getting P_ID: torrr001
Getting P_ID: dejop001
Getting P_ID: dever001
Getting P_ID: cronc002
Getting P_ID: duvaa001
Getting P_ID: kozmp001
Getting P_ID: barnt001
Getting P_ID: goldp001
Getting P_ID: cuevn001
Getting P_ID: andeb006
Getting P_ID: bauej001
Getting P_ID: rondj002
Getting P_ID: smoaj001
Getting P_ID: dixob001
Getting P_ID: swihb001
Getting P_ID: herrd002
Getting P_ID: willn001
Getting P_ID: nuner001
Getting P_ID: bourj002

In [11]:
run_batch(batch_3)

Starting..

Getting P_ID: dethb101
Getting P_ID: cox-l101
Getting P_ID: gomel101
Getting P_ID: metzr101
Getting P_ID: kendf101
Getting P_ID: doylb101
Getting P_ID: seilr101
Getting P_ID: alexm101
Getting P_ID: corrv101
Getting P_ID: valdj101
Getting P_ID: grotj101
Getting P_ID: weisg101
Getting P_ID: mccod101
Getting P_ID: smitj101
Getting P_ID: hernp101
Getting P_ID: crowd101
Getting P_ID: smitr102
Getting P_ID: ciasd101
Getting P_ID: smitk001
Getting P_ID: wiegt001
Getting P_ID: gomer001
Getting P_ID: sullm001
Getting P_ID: gerbc001
Getting P_ID: huntr001
Getting P_ID: hearj001
Getting P_ID: parda001
Getting P_ID: martm001
Getting P_ID: greeg001
Getting P_ID: joner001
Getting P_ID: dewij001
Getting P_ID: dowek001
Getting P_ID: jackk001
Getting P_ID: tabog001
Getting P_ID: palar001
Getting P_ID: higgm001
Getting P_ID: santa001
Getting P_ID: motaj001
Getting P_ID: wakad001
Getting P_ID: tackj001
Getting P_ID: lindd001
Getting P_ID: millj001
Getting P_ID: rossr001
Getting P_ID: zoske001

In [12]:
run_batch(batch_4)

Starting..

Getting P_ID: michc001
Getting P_ID: villo001
Getting P_ID: omalr001
Getting P_ID: sancj002
Getting P_ID: hampj001
Getting P_ID: albem001
Getting P_ID: kuo-h001
Getting P_ID: pereb001
Getting P_ID: morij001
Getting P_ID: canna001
Getting P_ID: mclem002
Getting P_ID: youms001
Getting P_ID: zitob001
Getting P_ID: chicm001
Getting P_ID: vandr001
Getting P_ID: garcf002
Getting P_ID: lerea001
Getting P_ID: speil001
Getting P_ID: durbj001
Getting P_ID: happj001
Getting P_ID: reyej003
Getting P_ID: jimeu001
Getting P_ID: livib001
Getting P_ID: petiy001
Getting P_ID: hanrj001
Getting P_ID: miscp001
Getting P_ID: harit001
Getting P_ID: barod001
Getting P_ID: moraf001
Getting P_ID: gutij002
Getting P_ID: castf002
Getting P_ID: pattt001
Getting P_ID: sheat001
Getting P_ID: cassj001
Getting P_ID: ennij001
Getting P_ID: seddc001
Getting P_ID: bullb001
Getting P_ID: paulf001
Getting P_ID: bennj002
Getting P_ID: blact001
Getting P_ID: humbp001
Getting P_ID: milla002
Getting P_ID: santj003